##### 0. Objective: Build a system where agents use Bayesian inference and fuzzy decision-making while communicating over MQTT. We can architect the system as follows:

1. Agent Structure: Each agent has:

* A Bayesian Network for reasoning under uncertainty.
* A Fuzzy Logic-based Decision System for actions based on the Bayesian inference.
* MQTT Communication for sending and receiving information to/from other agents.

2. Distributed Agent Network:

* Each agent subscribes to an MQTT topic relevant to its environment (e.g., temperature, humidity, etc.).
* Each agent processes its data with Bayesian inference, then uses fuzzy logic for decision-making.
* The agents exchange information over MQTT, allowing distributed, collaborative decision-making.

In [1]:
import logging
logging.getLogger('numexpr').setLevel(logging.WARNING)

In [2]:
import os
os.environ["NUMEXPR_MAX_THREADS"] = "8"

##### 1. Common Bayesian Network Structure
Each agent will have a Bayesian Network model to reason about its environment. Here we define a model for diagnosing a system based on sensor inputs.

In [3]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

def build_bayesian_network():
    # Create a Bayesian Network structure
    model = BayesianNetwork([('SensorFailure', 'Temperature'), ('SensorFailure', 'Humidity')])

    # Define CPDs
    cpd_sensor_failure = TabularCPD(variable='SensorFailure', variable_card=2, values=[[0.9], [0.1]])
    cpd_temperature = TabularCPD(variable='Temperature', variable_card=2, 
                                 values=[[0.8, 0.3], [0.2, 0.7]],
                                 evidence=['SensorFailure'], evidence_card=[2])
    cpd_humidity = TabularCPD(variable='Humidity', variable_card=2, 
                              values=[[0.7, 0.2], [0.3, 0.8]],
                              evidence=['SensorFailure'], evidence_card=[2])

    # Add CPDs to the model
    model.add_cpds(cpd_sensor_failure, cpd_temperature, cpd_humidity)

    # Check model validity
    assert model.check_model()

    return model

# Bayesian inference function
def bayesian_inference(model, temperature, humidity):
    inference = VariableElimination(model)
    evidence = {'Temperature': temperature, 'Humidity': humidity}
    result = inference.query(variables=['SensorFailure'], evidence=evidence)
    return result.values[1]  # Return probability of sensor failure

##### 2. Fuzzy Logic System for Decision Making
Agents use fuzzy logic to make decisions based on the Bayesian inference results.

In [4]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

def build_fuzzy_system():
    # Create fuzzy variables
    prob_sensor_failure = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'prob_sensor_failure')
    action_level = ctrl.Consequent(np.arange(0, 101, 1), 'action_level')

    # Define fuzzy membership functions
    prob_sensor_failure['low'] = fuzz.trimf(prob_sensor_failure.universe, [0, 0, 0.5])
    prob_sensor_failure['medium'] = fuzz.trimf(prob_sensor_failure.universe, [0.3, 0.5, 0.7])
    prob_sensor_failure['high'] = fuzz.trimf(prob_sensor_failure.universe, [0.5, 1, 1])

    action_level['low'] = fuzz.trimf(action_level.universe, [0, 0, 50])
    action_level['moderate'] = fuzz.trimf(action_level.universe, [25, 50, 75])
    action_level['high'] = fuzz.trimf(action_level.universe, [50, 100, 100])

    # Define rules
    rule1 = ctrl.Rule(prob_sensor_failure['low'], action_level['low'])
    rule2 = ctrl.Rule(prob_sensor_failure['medium'], action_level['moderate'])
    rule3 = ctrl.Rule(prob_sensor_failure['high'], action_level['high'])

    # Create control system and simulation
    action_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
    action_sim = ctrl.ControlSystemSimulation(action_ctrl)

    return action_sim

# Fuzzy decision-making function
def fuzzy_decision(action_sim, prob_failure):
    action_sim.input['prob_sensor_failure'] = prob_failure
    action_sim.compute()
    return action_sim.output['action_level']

##### 3. MQTT Communication Setup
Each agent will use MQTT to publish and subscribe to messages from other agents

In [5]:
import paho.mqtt.client as mqtt

# MQTT communication setup
def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    client.subscribe("sensor/temperature")
    client.subscribe("sensor/humidity")

def on_message(client, userdata, msg):
    print(f"Received message: {msg.topic} {str(msg.payload)}")

def publish_data(client, topic, data):
    client.publish(topic, data)

def mqtt_setup(agent_id):
    client = mqtt.Client(client_id=agent_id)  # Explicitly set client_id as keyword
    client.on_connect = on_connect
    client.on_message = on_message
    client.connect("mqtt.eclipseprojects.io", 1883, 60)  # Use a public broker like Eclipse
    return client

##### 4. Agent Implementation
Now we can combine the Bayesian Network, fuzzy logic, and MQTT communication into a complete agent.

In [7]:
import time

def run_agent(agent_id, initial_temperature, initial_humidity):
    # Build Bayesian Network and Fuzzy System
    bayesian_model = build_bayesian_network()
    fuzzy_system = build_fuzzy_system()

    # Set up MQTT communication
    client = mqtt_setup(agent_id)

    # Simulate periodic sensor data updates
    while True:
        # Bayesian inference for sensor failure
        prob_failure = bayesian_inference(bayesian_model, initial_temperature, initial_humidity)

        # Fuzzy logic for decision making
        action_level = fuzzy_decision(fuzzy_system, prob_failure)

        # Publish data and decisions to MQTT topics
        publish_data(client, f"agent/{agent_id}/sensor_failure_probability", prob_failure)
        publish_data(client, f"agent/{agent_id}/action_level", action_level)

        print(f"Agent {agent_id}: Prob Failure: {prob_failure:.2f}, Action Level: {action_level:.2f}")

        # Simulate a time delay between messages
        time.sleep(5)

    client.loop_forever()

C:\Users\luism\AppData\Local\Temp\ipykernel_17688\2803183985.py:16: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(client_id=agent_id)  # Explicitly set client_id as keyword


Agent agent_1: Prob Failure: 0.10, Action Level: 17.22
Agent agent_1: Prob Failure: 0.10, Action Level: 17.22
Agent agent_1: Prob Failure: 0.10, Action Level: 17.22
Agent agent_1: Prob Failure: 0.10, Action Level: 17.22
Agent agent_1: Prob Failure: 0.10, Action Level: 17.22
Agent agent_1: Prob Failure: 0.51, Action Level: 50.68
Agent agent_1: Prob Failure: 0.10, Action Level: 17.22
Agent agent_1: Prob Failure: 0.51, Action Level: 50.68
Agent agent_1: Prob Failure: 0.10, Action Level: 17.22
Agent agent_1: Prob Failure: 0.10, Action Level: 17.22
Agent agent_1 completed its iterations.
Connected with result code 0


KeyboardInterrupt: 

##### 5. Execution

A. Each agent runs the run_agent() function, which:

* Continuously gathers data (temperature, humidity).
* Computes the probability of sensor failure using the Bayesian Network.
* Makes decisions using fuzzy logic based on the probability.
* Publishes the results via MQTT for other agents to read and use for collaboration.

B. You can run multiple agents in separate processes or on different machines, as long as they connect to the same MQTT broker.

##### 6. Final Thoughts

* This system architecture allows agents to make decisions based on probabilistic reasoning and fuzzy logic while exchanging data with other agents in a distributed, collaborative environment.
* MQTT ensures lightweight communication, suitable for IoT-like distributed systems.